# The Number Partition Problem

[The Number Partition Problem](https://en.wikipedia.org/wiki/Partition_problem)
is famous computational problem that asks you to *partition* a list of numbers
into two lists such that the absolute value of the difference of the sums of the
numbers in the two lists is as small as possible. The best possible difference
is 0, but this is not always possible.

The term *partition* means that each number of the original list is put into
exactly one of the other two lists.

The problem is [NP-complete](https://en.wikipedia.org/wiki/NP-complete), which
means that the best algorithms for solving it take exponential time. However, it
is unknown whether there is a polynomial-time algorithm for the problem.

Before looking at a couple of algorithms for solving the problem, lets' write some code to read numbers from a file and compute their sum.

## Question 1: Getting Numbers from a File

Write a function call `get_numbers(filename)` that returns a list of the numbers
in the test file `filename`. Assume that `filename` is a text file with one int
on each line.

For example, suppose the file `scores.txt` contains this:

```
4
9
1
1
0
```

Then calling `get_numbers('scores.txt')` returns the list `[4, 9, 1, 1, 10]`.

### Question 2: Summing the Numbers in a File

Write a function call `get_filesum(filename)` that returns the sum of the
numbers in the file `filename`. Assume that `filename` is a text file with one
int on each line.

Use the `get_numbers(filename)` function to help implement this function.

For example, suppose the file `scores.txt` contains this:

```
4
9
1
1
0
```

Then calling `get_filesum('scores.txt')` returns `15`.

### Question 3: Writing Two Output Files

Write a function called `write_lists(left, right)` that takeslists `left` and
`right` as input and writes their contents to the files `left.txt` and
`right.txt`, respectively.

For example, calling `write_lists([1, 2, 3], [4, 5, 6])` should create the files
`left.txt` and `right.txt` with the following contents. `left.txt` is:

```
1
2
3
```

And `right.txt` is:

```
4
5
6
```

## Question 4: Scoring Two Lists

Write a function called `get_score(left, right)` that returns the absolute value
of the difference of the sums of the numbers in the lists `left` and `right`.

For example, suppose `left` is `[1, 2, 3]` and `right` is `[4, 5, 6]`. Then
calling `get_score(left, right)` should return `9`.

## Question 5: Method 1 for Partitioning the Numbers

One algorithm for solving the number partition problem is the following:

- set *left* to the empty list
- set *right* to the empty list
- for each number in the original list:
  - add the number to the list with the current smallest sum (in case of a tie
    always choose *left*)

Write a function called `split_numbers_method1(numbers)` that implements this
algorithm.

For example, `split_numbers_method1([1, 2, 3, 4, 5, 6])` should return `([2, 4,
6], [1, 3, 5])`, which has a score of `abs(12 - 9) = 3`.

**This method does *not* necessarily return the optimal solution!** The order of
the numbers in the original list makes a difference. For example,
`split_numbers_method1([1, 2, 3, 4, 5, 6])` returns `([1, 3, 5], [2, 4, 6])`,
for a score of `abs(9 - 12) = 3`. But if you reverse the order, then
`split_numbers_method1([6, 5, 4, 3, 2, 1])` returns `([6, 3, 2], [5, 4, 1])`,
which has a better score of `abs(11 - 10) = 1`.

Note that in these examples the two returned lists are the same length. That's
just a coincidence. In general, the returned lists can have different lengths.
For example, `split_numbers_method1([5, 20, 14, 1])` returns `([5, 14, 1],
[20])`, which has a perfect score of `abs(20 - 20) = 0`.

## Question 6: Variations of Method 1

Implement the following variations of methods 1:

- `split_numbers_method1_sort_ascending(numbers)` Sorts the numbers in
  *ascending* order (smallest to biggest) and then calls
  `split_numbers_method1`.
- `split_numbers_method1_sort_descending(numbers)` Sorts the numbers in
  *descending* order (biggest to smallest) and then calls
  `split_numbers_method1`.
- `split_numbers_method1_sort_ascending_randomized(numbers)` Before calling
  `split_numbers_method1`, randomly shuffle the order of the numbers. Using the
  `shuffle` function from the `random` module.
- `split_numbers_method1_sort_ascending_randomized(numbers, n)` Call the
  previous method `n` times and return the best result.

For example, `split_numbers_method1_sort_ascending_randomized([1, 2, 3, 4, 5, 6])`
might return `([1, 3, 5], [2, 4, 6])`, which has a score of `abs(9 - 12) = 3`.


## Question 7: Method 2 for Partitioning the Numbers

For this question, your task is to find the largest list length for which
`split_numbers_method2(nums)` runs in less than a minute.

`split_numbers_method2(nums)` *partitions* the numbers into all possible splits
of two lists and then selects the partition with the smallest score. Thus it
guarantees to find the optimal solution. Unfortunately, it is slow for even
moderately large lists. 

The problem is that the number of partitions grows *exponentially* with the size
of the input list, i.e. a list of length $n$ has about $2^n$ partitions. So, for
instance, a list of length 20 has about $2^{20} = 1048576$ partitions (a little
over a million), a list of 21 numbers has twice as many (about 2 million
partitions), and so on. Every time the length of the list increases *by 1*, the
number of partitions *doubles*.

Read the `all_partitions` function below. It returns a list of all possible
partitions of a list into two lists. It does this *recursively*. To generate all
partitions of $n$ items, it first generates a list of all partitions of $n-1$
items. Then it replaces each individual partitions by two new partitions,
generated by adding $n$ to the left or right list of the partition.

In [10]:
def all_partitions(nums):
    """Returns a list of all possible partitions of the list nums into two lists.
    """
    if len(nums) == 0:
        return []
    elif len(nums) == 1:
        left = [nums[0]]
        right = []
        return [ [left, right] ]
    else:
        # call the first element of numns n
        n = nums[0]
        # recursively call all_partitions on the rest of the numbers
        parts = all_partitions(nums[1:])
        # initialize an empty list to store the results
        result = []
        # loop over all partitions and replace each one with two new partitions, each containing n
        for part in parts:
            left, right = part[0], part[1]
            result.append([ [n] + left,       right ])
            result.append([       left, [n] + right ])
            
        return result

def split_numbers_method2(nums):
    # generate all partitions of the numbers into two lists
    all_parts = all_partitions(nums)
    
    # initialize the best partition and its score
    best_left = all_parts[0][0]
    best_right = all_parts[0][1]
    best_score = get_score(best_left, best_right)
    
    # loop over all partitions and find the one with the smallest score
    for part in all_parts:
        left, right = part[0], part[1]
        score = get_score(left, right)
        if score < best_score:
            best_left, best_right = left, right
            best_score = score
            
    return best_left, best_right

best_left, best_right = split_numbers_method2([4, 1, 2, 2, 6, 1, 5])
print(f'best left: {best_left}')
print(f'best right: {best_right}')
print(f'score: {get_score(best_left, best_right)}')

best left: [1, 2, 2, 1, 5]
best right: [4, 6]
score: 1


## Question 8: Estimating Running Time of 100 Numbers

The file [nums100.txt](nums100.txt) has 100 numbers. If you were to run
`split_numbers_method2` on those numbers, how long do you estimate it would take
for the function to return an answer? Estimate your answer using the data from
the previous question.

## Question 9: Partitioning 100 Numbers

Using the approximation could from question 6, what is the lowest-scoring
partition you can find for [nums100.txt](nums100.txt)?